# Clasificación de Texto Usando Embeddings Personalizados

Este notebook presenta la creación de un dataset de oraciones con su respectivo autor, con el que luego se entrenan varios clasificadores usando variaciones de redes feed-forward y distintos embeddings pre-entrenados.

## 0. Importación de Librerías

In [1]:
import re

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

## 1. Creación del Dataset

Primero, creamos el dataset de oraciones etiquetadas según el autor.

In [2]:
def load_raw_data(file_path: str) -> str:
    """
    Carga el texto crudo a partir de un archivo de texto
    
    Args:
    file_path (str): Ruta del archivo de texto.
    
    Returns:
    str: Texto crudo.
    """
    # Leer el texto crudo
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    return text

def extract_sentences(book: str) -> list[str]:
    """
    Extrae extractos de un libro asegurando que cumplan con ciertas condiciones de tamaño
    
    Args:
    book (str): Texto crudo.
    
    Returns:
    list[str]: Lista de extractos del libro.
    """
    # Separar el texto en bloques usando líneas completamente vacías como delimitadores
    lines = book.split('***')[2].split('\n\n')

    # Eliminar espacios en blanco al inicio y al final de cada línea
    lines = [line.strip() for line in lines]

    # Eliminar lineas vacias y títulos no relevantes
    lines = [line for line in lines if line and not line.startswith('CHAPTER') and not line.startswith('[Illustration]')]

    # Eliminar saltos de líneas de las oraciones
    lines = [line.replace('\n', ' ') for line in lines]

    # Usar word count para extraer las oraciones
    sentences = []
    for sentence in lines:
        if len(sentence.split()) > 250:
            sentences.extend(sent_tokenize(sentence))  # Dividir en oraciones si es demasiado largo
        else:
            sentences.append(sentence)

    # Filtrar por oraciones que tienen entre 150 y 250 palabras
    sentences = [sentence for sentence in sentences if 150 <= len(sentence.split()) <= 250]

    # Eliminar espacios en blanco al inicio y al final de cada línea nuevamente
    sentences = [sentence.strip() for sentence in sentences]

    return sentences

In [3]:
# Ruta a los libros originales junto con su autor
raw_books = {
    'austen_sense-and-sensibility': {
        'file_path': 'data/raw/austen_sense-and-sensibility.txt',
        'author': 'Jane Austen',
    },
    'austen_pride-and-prejudice': {
        'file_path': 'data/raw/austen_pride-and-prejudice.txt',
        'author': 'Jane Austen',
    },
    'austen_emma': {
        'file_path': 'data/raw/austen_emma.txt',
        'author': 'Jane Austen',
    },
    'tolstoy_youth': {
        'file_path': 'data/raw/tolstoy_youth.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_war-and-peace': {
        'file_path': 'data/raw/tolstoy_war-and-peace.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_anna-karenina': {
        'file_path': 'data/raw/tolstoy_anna-karenina.txt',
        'author': 'Leo Tolstoy',
    },
    'joyce_dubliners': {
        'file_path': 'data/raw/joyce_dubliners.txt',
        'author': 'James Joyce',
    },
    'joyce_a-portrait-of-the-artist-as-a-young-man': {
        'file_path': 'data/raw/joyce_a-portrait-of-the-artist-as-a-young-man.txt',
        'author': 'James Joyce',
    },
    'joyce_ulysses': {
        'file_path': 'data/raw/joyce_ulysses.txt',
        'author': 'James Joyce',
    }
}

In [4]:
# Crear dataframe con las oraciones extraídas de los libros
df = pd.DataFrame(columns=['author', 'sentence'])

# Extraer oraciones para cada libro y concatenarlas en el dataframe
for book in raw_books.values():
    corpus = load_raw_data(book['file_path'])
    author = book['author']
    
    # Extraer oraciones
    sentences = extract_sentences(corpus)

    df = pd.concat([df, pd.DataFrame({'author': author, 'sentence': sentences})], ignore_index=True)

df.head()

,author,sentence
0,Jane Austen,The family of Dashwood had long been settled i...
1,Jane Austen,"The old gentleman died: his will was read, and..."
2,Jane Austen,"No sooner was his father’s funeral over, than ..."
3,Jane Austen,"“Certainly not; but if you observe, people alw..."
4,Jane Austen,Edward Ferrars was not recommended to their go...


In [5]:
# Guardar dataset como archivo CSV
df.to_csv('data/classifier/sentences.csv', index=False)

In [6]:
# Contar el número de datos por autor
author_counts = df['author'].value_counts()

# Crear DataFrame resumen
summary_df = author_counts.reset_index()
summary_df.columns = ['author', 'num_training_data']

summary_df

,author,num_training_data
0,Leo Tolstoy,866
1,Jane Austen,426
2,James Joyce,321


## 2. Preprocesamiento del Dataset

Preprocesamos el dataset separandolo en entrenamiento y prueba. Adicionalmente, tokenizamos el texto para poder mapear las palabras a los embeddings construidos y usarlos como la capa de entrada de los modelos de redes neuronales.

In [7]:
# Dividir en conjunto de entrenamiento, validación y prueba
x_train, x_temp, y_train, y_temp = train_test_split(df['sentence'], df['author'], train_size=0.7, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size=0.5, random_state=42)

# Tokenización usando Keras
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

# Convertir el texto en secuencias de enteros
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Rellenar las secuencias para que tengan la misma longitud
max_length = max([len(seq) for seq in x_train_seq])
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_val_pad = pad_sequences(x_val_seq, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post')

## 3. Definición de los Modelos de Redes Neuronales

Cargamos los los embeddings de Word2Vec pre-entrenados, creamos las capas de embeddings a partir de ellos, y definimos los tres tipos de arquitecturas de redes neuronales que usaremos.

In [8]:
# Ruta a los modelos Word2Vec combinados con diferentes tamaños de vectores
books_models = [
    'data/models/Books_50_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model',
    'data/models/Books_100_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model',
    'data/models/Books_300_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba.model'
]

# Cargar los embeddings de Word2Vec pre-entrenados
word2vec_model_50 = gensim.models.Word2Vec.load(books_models[0])
word2vec_model_100 = gensim.models.Word2Vec.load(books_models[1])
word2vec_model_300 = gensim.models.Word2Vec.load(books_models[2])

In [9]:
def create_embedding_layer(word2vec_model, tokenizer, max_length):
    """
    Crea una capa de embeddings a partir de un modelo Word2Vec y un tokenizer.

    Args:
    word2vec_model: Modelo Word2Vec preentrenado.
    tokenizer: Tokenizer que contiene el índice de palabras.
    max_length (int): Longitud máxima de las secuencias de entrada.

    Returns:
    Embedding: Capa de embedding de Keras que utiliza la matriz de embeddings generada.
    """
    # Crear la matriz de embeddings para el modelo Word2Vec
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]

    # Definir la capa de embedding en Keras
    embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1,
                                output_dim=word2vec_model.vector_size,
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=False)
    
    return embedding_layer

# Crear las capas de embeddings a partir de los modelos Word2Vec
embedding_layer_50 = create_embedding_layer(word2vec_model_50, tokenizer, max_length)
embedding_layer_100 = create_embedding_layer(word2vec_model_100, tokenizer, max_length)
embedding_layer_300 = create_embedding_layer(word2vec_model_300, tokenizer, max_length)

In [10]:
# Arquitectura 1: Modelo sencillo
def create_ffnn_model_1(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward simple.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Salida con 3 clases (autores)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Arquitectura 2: Modelo con más capas
def create_ffnn_model_2(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más capas.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Arquitectura 3: Modelo con más unidades
def create_ffnn_model_3(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más unidades.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## 4. Creación y Evaluación de los Modelos de Redes Neuronales

Creamos un modelo con cada tipo de arquitectura y capa de embeddings y evaluamos su accuracy, precision y recall.

In [11]:
# Codificación de etiquetas (autores)
label_encoder = LabelEncoder()
y_train_encoded = to_categorical(label_encoder.fit_transform(y_train))
y_val_encoded = to_categorical(label_encoder.transform(y_val))
y_test_encoded = to_categorical(label_encoder.transform(y_test))

In [12]:
def evaluate_model(model, x_test_pad, y_test_encoded):
    """
    Evalúa el rendimiento de un modelo entrenado calculando accuracy, precision y recall.
    
    Args:
    model (keras.models.Model): El modelo entrenado.
    x_test_pad (numpy.ndarray): Conjunto de datos de prueba preprocesados y tokenizados.
    y_test_encoded (numpy.ndarray): Etiquetas de prueba codificadas en formato one-hot.
    
    Returns:
    tuple: Un tupla que contiene:
        - accuracy (float): La proporción de predicciones correctas.
        - precision (float): La proporción de predicciones positivas correctas (precisión macro).
        - recall (float): La proporción de verdaderos positivos detectados (recall macro).
    """
    # Obtener predicciones del modelo
    y_pred = model.predict(x_test_pad)
    
    # Convertir las predicciones y etiquetas de one-hot a clases
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test_encoded, axis=1)
    
    # Calcular accuracy
    accuracy = np.mean(y_pred_classes == y_test_classes)
    
    # Calcular precisión y recall usando la métrica macro (promedio entre todas las clases)
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    
    return accuracy, precision, recall

In [13]:
# Iterar sobre las capas de embeddings y las dimensiones de los modelos Word2Vec (50, 100, 300 dimensiones)
for embedding_layer, dimensions in [(embedding_layer_50, 50), (embedding_layer_100, 100), (embedding_layer_300, 300)]:
    
    # Iterar sobre las funciones de creación de modelos FFNN (modelos 1, 2 y 3)
    for i, model_fn in enumerate([create_ffnn_model_1, create_ffnn_model_2, create_ffnn_model_3], 1):
        
        # Crear el modelo usando la capa de embeddings actual
        print(f"\nEntrenando Modelo {i} con {dimensions} dimensiones..." "\n")
        model = model_fn(embedding_layer)
        
        # Mostrar el resumen del modelo (capas y dimensiones)
        model.summary()

        # Entrenar el modelo con el conjunto de datos de entrenamiento y validar con el conjunto de validación
        history = model.fit(x_train_pad, y_train_encoded, 
                            epochs=10, batch_size=32, 
                            validation_data=(x_val_pad, y_val_encoded), 
                            verbose=1)
        
        # Evaluar el modelo en el conjunto de prueba
        accuracy, precision, recall = evaluate_model(model, x_test_pad, y_test_encoded)

        # Mostrar los resultados finales de la evaluación (accuracy, precision y recall)
        print(f"\nEvaluación del Modelo {i} con embeddings de {dimensions} dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}", "\n")


Entrenando Modelo 1 con 50 dimensiones...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 254, 50)           851050    
                                                                 
 flatten (Flatten)           (None, 12700)             0         
                                                                 
 dense (Dense)               (None, 128)               1625728   
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 2,477,165
Trainable params: 1,626,115
Non-trainable params: 851,050
_________________________________________________________________
Epoch 1/10
36/36 [==============================] - 1s 19ms/step - loss: 0.6705 - accuracy: 0.7192 - val_loss: 0.4193 - val_accu